In [1]:
!pip install autoawq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 53.3 MB/s eta 0:00:00:00:0100:01


In [19]:
from huggingface_hub import notebook_login as hfl
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datetime import datetime
from pynvml import *
import awq
import torch

In [4]:
hfl()

In [9]:
def printGPUState():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    
    print(f"GPU Memory Occupied: {info.used // 1024 ** 2} MB.")

In [10]:
%%time
model_id = "0xVolt/Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit"

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    device_map={"": 0}
)

printGPUState()

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

GPU Memory Occupied: 5967 MB.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 4.68 s, sys: 2.07 s, total: 6.75 s
Wall time: 23 s


In [36]:
def generateOutputWithModelPipeline(model, tokenizer, prompt, temperature=0.7) : 
    pipe = pipeline(
        task="text-generation", 
        model=model, 
        tokenizer=tokenizer
    )
    
    start = datetime.now()
    
    output = pipe(
        prompt,
        do_sample=True,
        max_new_tokens=3000, 
        temperature=temperature, 
        top_k=50, 
        top_p=0.95,
        num_return_sequences=1
    )
    
    stop = datetime.now()
    
    totalTimeToPrompt = stop - start
    print(f"Execution Time : {totalTimeToPrompt}")
    
    return output

In [37]:
prompt = """
Write a function in C++ that appends an element to the end of a linked list. Explain your code with comments as necessary.
"""

In [38]:
%%time
output = generateOutputWithModelPipeline(model, tokenizer, prompt)
print(output["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Execution Time : 0:54:56.503868


TypeError: list indices must be integers or slices, not str

In [39]:
output

[{'generated_text': '\nWrite a function in C++ that appends an element to the end of a linked list. Explain your code with comments as necessary.\n```cpp\n#include<iostream>\nusing namespace std;\n\n// Structure for a node in the linked list\nstruct Node {\n    int data;\n    Node* next;\n};\n\n// Function to insert a new node at the end of the linked list\nvoid append(Node*& head, int newData) {\n    // Create a new node with the given data\n    Node* newNode = new Node();\n    newNode->data = newData;\n    newNode->next = nullptr;\n\n    // If the linked list is empty, set the new node as the head\n    if (*head == nullptr) {\n        *head = newNode;\n    } else {\n        // Traverse the linked list to find the last node\n        Node* last = *head;\n        while (last->next!= nullptr) {\n            last = last->next;\n        }\n\n        // Append the new node to the last node\'s next\n        last->next = newNode;\n    }\n}\n\nint main() {\n    // Create an empty linked list\n

#### 